In [ ]:
from agents.agent import run_agent

import warnings
from dotenv import load_dotenv

load_dotenv("plash.env")
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

Things to remember to talk about
- prompt caching
- function calling
- reasonign models - thought tokens - need to be passed back - thinking traces need to be passed back
- LLM agent abstractions - less hackable?
- litellm
- advanced shit - https://docs.litellm.ai/blog/anthropic_advanced_features#tool-search
- https://www.anthropic.com/engineering/advanced-tool-use
- dont want a lot of tools
- progressive disclosure - tool search 
- https://docs.litellm.ai/blog/gemini_3_flash
- https://docs.litellm.ai/blog/anthropic_advanced_features
- https://github.com/openai/openai-agents-python
- https://www.anthropic.com/engineering/multi-agent-research-system
- context - tools blow it up
- they all have diff syntax
- Anthropic new beta runner ---> https://platform.claude.com/docs/en/agents-and-tools/tool-use/implement-tool-use

In [ ]:
for rec in run_agent(
    [
        {
            "role": "user",
            "content": "Get the weather fo 4 cities of your choice. If all are the same then do a fifth. Then get the average using the coding tool.",
        }
    ]
):
    print(rec)

In [ ]:
from litellm import completion
from dotenv import load_dotenv

load_dotenv()

response = completion(
    model="gemini/gemini-3-flash-preview",
    messages=[{"role": "user", "content": "Hello!"}],
    reasoning_effort="low",
)

print(response.choices[0].message.content)

In [ ]:
import litellm

litellm.supports_function_calling(model="claude-opus-4-5-20251101")

In [8]:
from litellm import completion
import httpx
import base64
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

load_dotenv('plash.env')

def test_litellm(model):
    # First call - ask model to use the tool
    tool_schema = {'type': 'function',
        'function': {'name': 'get_image','description': 'Returns a test image',
        'parameters': {'type': 'object', 'properties': {}, 'required': []}}
        }   
    msgs = [{"role": "user", "content": "Please call the get_image tool and describe the image it returns."}]
    r = completion(model, msgs, tools=[tool_schema])

    # Get the tool call info
    tc = r.choices[0].message.tool_calls[0]
    msgs.append(r.choices[0].message)

    # Download and encode image
    img_data = httpx.get("https://raw.githubusercontent.com/AnswerDotAI/lisette/refs/heads/main/nbs/samples/puppy.jpg").content
    encoded = base64.b64encode(img_data).decode("utf-8")

    # Tool result with image
    msgs.append({
        "role": "tool",
        "tool_call_id": tc.id,
        "content": [{'type': 'image_url', 'image_url': f'data:image/jpeg;base64,{encoded}'}]
    })

    # Second call with image result
    r2 = completion(model, msgs, tools=[tool_schema])
    print(r2.choices[0].message.content)



In [ ]:
test_litellm("gemini/gemini-3-flash-preview") # works

ariThis image shows a close-up of a Cavalier King Charles Spaniel puppy lying on a patch of grass.

The puppy has classic "Blenheim" markings, with a white coat and large chestnut-brown patches over its ears and around its dark, expressive eyes. It has a white muzzle and a small black nose. Its long, wavy ears frame its face, and its front paw is stretched forward onto the grass.

To the left of the puppy is a lush green bush filled with many small, light purple daisy-like flowers. The puppy is looking directly toward the camera with a gentle expression. The background is slightly out of focus, suggesting an outdoor setting near a stone or wooden structure.


In [10]:
test_litellm('claude-opus-4-5-20251101') # works

This is an adorable image of a Cavalier King Charles Spaniel puppy! The puppy has the classic Blenheim coloring with a white coat and chestnut (reddish-brown) markings on its ears and around its eyes. 

The puppy is lying on green grass and has a sweet, innocent expression with big, dark eyes looking directly at the camera. Its ears are long and floppy with wavy fur, characteristic of the breed.

In the background, there's a lovely cluster of purple/lavender aster flowers, which creates a beautiful contrast with the puppy's coloring. There also appears to be part of a wooden structure (possibly a bench or planter) visible on the right side of the image.

It's a charming, heartwarming photo that captures the gentle and endearing nature of this popular companion breed.


In [12]:
test_litellm('gpt-5.2') # raises error


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



BadRequestError: litellm.BadRequestError: OpenAIException - Invalid type for 'messages[2].content[0].image_url': expected an object, but got a string instead.

In [13]:
from litellm import completion
import httpx
import base64
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")
load_dotenv("plash.env")

def test_litellm(model):
    tool_schema = {
        "type": "function",
        "function": {
            "name": "get_image",
            "description": "Returns a test image",
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
    }

    msgs = [{
        "role": "user",
        "content": "Please call the get_image tool and describe the image it returns."
    }]

    r = completion(model, msgs, tools=[tool_schema])

    tc = r.choices[0].message.tool_calls[0]
    msgs.append(r.choices[0].message)

    img_data = httpx.get(
        "https://raw.githubusercontent.com/AnswerDotAI/lisette/refs/heads/main/nbs/samples/puppy.jpg"
    ).content
    encoded = base64.b64encode(img_data).decode("utf-8")

    # 1) Tool outputs must be plain text/JSON for OpenAI Chat Completions
    msgs.append({
        "role": "tool",
        "tool_call_id": tc.id,
        "content": "Fetched image successfully. Image will be provided in the next user message."
    })

    # 2) Provide the image as a *user* multimodal message
    # NOTE: OpenAI expects image_url to be an object: {"url": "..."}
    msgs.append({
        "role": "user",
        "content": [
            {"type": "text", "text": "Here is the image returned by get_image. Please describe it."},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded}"}},
        ],
    })

    r2 = completion(model, msgs, tools=[tool_schema])
    print(r2.choices[0].message.content)



In [14]:
test_litellm("gemini/gemini-3-flash-preview")   # should still work



This image shows an adorable Cavalier King Charles Spaniel puppy lying in the grass next to a bush of purple flowers.

The puppy has classic "Blenheim" markings, featuring a white face and body with large, chestnut-colored patches over its long, wavy ears and around its dark, expressive eyes. It is lying down with its front paws stretched out towards the camera, looking directly forward with a soft expression.

To the left of the puppy is a lush green plant covered in small, daisy-like purple flowers with yellow centers. The scene is set outdoors on a lawn, with soft, natural lighting that highlights the texture of the puppy's fur and the surrounding greenery.


In [15]:
test_litellm("claude-opus-4-5-20251101")        # should still work


This is an adorable image of a **Cavalier King Charles Spaniel puppy**! 

The puppy has the breed's characteristic markings with a beautiful chestnut/reddish-brown and white coat (known as the "Blenheim" coloring). It has soulful, dark eyes with an sweet, innocent expression typical of young puppies.

The puppy is lying down on green grass, with its white front paws stretched out in front. Behind the puppy, there's a lovely bush of purple/lavender aster flowers that creates a beautiful, colorful backdrop. The setting appears to be a garden on a nice day, giving the photo a charming, pastoral feel.

It's a heartwarming and picturesque image that really captures the cuteness and gentle nature of this beloved breed!


In [16]:
test_litellm("openai/gpt-5.2")                  # this is the pattern OpenAI accepts

The image shows a small puppy lying on grass, facing the camera. It has a white face with chestnut-brown ears and markings, large dark eyes, and one front paw stretched forward. To the left of the puppy is a cluster of small purple flowers, and there appears to be a wooden surface or step in the background.
